In [1]:
# start with tech, then for all entries in my database 

In [7]:
# After modeling, 
# Other possible features to extract _____________________________
# - n pictures -- use beautiful soup
# - has video
# - project body length 
# - project n misspelligns
# - text quality, sentiment analysis
# - other reward parameters

# lower priority -- 
# - international projects


In [8]:
# Features I already have _______________________________________
# - rewards: n, min, max, intn shipping
# - n creator backed
# - title word/char len
# - blurb word/char len
# - launch time (in Unix time)
# - project duration
# - goal

# I could add easily 
# - day of the month
# - month
# - time between launch and ship of rewards


In [9]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
import json

In [10]:
from extract_kickstarter_features import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [12]:
with con:
    
    cur = con.cursor()
    cur.execute("SHOW TABLES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('Creator',)
('Project_info',)
('Rewards',)
('Scraped_pages',)


In [ ]:
tech_subset_ids = 

In [14]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Id FROM Project_info WHERE Currency='USD' \
                    AND Category_slug LIKE 'tech%' AND Project_date != 'live'")
    ids = cur.fetchall()
    cur.close()

In [25]:
tech_subset_ids = np.asarray(ids)

In [29]:
n_projects = np.shape(tech_subset_ids)[0]

In [41]:
cols = ['id','pledged','goal','n rewards','min reward','max reward','intn ship',\
        'n creator backed','title n chars','blurb n chars','launch time','campaign dur']

In [74]:
tech_df = pd.DataFrame(index=range(n_projects),columns=cols)

In [75]:
tech_df.head(2)

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
# now loop through each project and collect raised and predictors -- later vectorize this?
for i,id_array in enumerate(tech_subset_ids):
    primary_key = id_array[0]
    if not i%200:
        print i,primary_key
    tech_df.ix[i,'id'] = primary_key
    
    
    with con:
        cur = con.cursor()
                            # 0     1        2      3      4             5
        query_txt = "SELECT Goal, Pledged, Name, Blurb, Launch_time, End_time \
                    FROM Project_info WHERE Id = " + str(primary_key)
        cur.execute(query_txt)
        
        rows = cur.fetchall()
        project_details = rows[0]
        tech_df.ix[i,'goal'] = project_details[0]
        tech_df.ix[i,'pledged'] = project_details[1]
        tech_df.ix[i,'launch time'] = project_details[4]
        tech_df.ix[i,'campaign dur'] = project_details[5] - project_details[4]
        tech_df.ix[i,'title n chars'] = len(project_details[2])
        tech_df.ix[i,'blurb n chars'] = len(project_details[3])
        cur.close()
    
    
        # now get rewards details 
        cur = con.cursor()
                            # 0           1             2        3                   
        query_txt = "SELECT N_rewards, Min_reward, Max_reward, Ships_intn \
                    FROM Rewards WHERE Id = " + str(primary_key)
        cur.execute(query_txt)
        rows = cur.fetchall()
        rewards_details = rows[0]
        tech_df.ix[i,'n rewards'] = rewards_details[0]
        tech_df.ix[i,'min reward'] = rewards_details[1]
        tech_df.ix[i,'max reward'] = rewards_details[2]
        tech_df.ix[i,'intn ship'] = rewards_details[3]
        cur.close()
        
        #get n campaigns creator backed
        cur = con.cursor()
                            # 0                              
        query_txt = "SELECT N_backed FROM Creator WHERE Id = " + str(primary_key)
        cur.execute(query_txt)
        rows = cur.fetchall()
        
        tech_df.ix[i,'n creator backed'] = rows[0][0]
        cur.close()
        
        
    

0 8018
200 8257
400 8489
600 8728
800 8967
1000 9204
1200 9431
1400 9667
1600 9902
1800 10119
2000 10368
2200 10641
2400 10895
2600 11157
2800 11421
3000 11683
3200 11935
3400 12184
3600 12453
3800 12703
4000 12954
4200 13209
4400 13468
4600 13732
4800 13986
5000 14249
5200 14516
5400 14781
5600 15050
5800 15332
6000 15590
6200 15868
6400 16145
6600 16424


In [80]:
tech_df.tail()

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur
6628,16461,25,100000,5,25,10000,-1,0,35,134,1414879251,2595600
6629,16462,15790,3500,8,59,250,1,1,36,132,1411511254,2592000
6630,16468,8904,5000,7,1,100,1,0,56,129,1411686079,3459600
6631,16469,615,50000,8,1,1000,0,0,51,103,1409441823,5184000
6632,16470,585549,7500,11,1,1337,1,2,44,114,1413159586,2595600


In [81]:
tech_df.to_pickle('tech_df.save')

In [82]:
np.shape(tech_df)

(6633, 12)